In [ ]:
import datetime
import pandas as pd
import numpy as np

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3.common.logger import configure
from finrl.meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../FinRL")

import itertools

import wandb
wandb.login(key="fd2d0476c22a97c32eef81b22402a4c73ab32831")


from finrl import config
from finrl import config_tickers
import os
#os.environ["CUDA_VISIBLE_DEVICES"]="0"

from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])


#超参数优化

sweep_config = {
    'method': 'random'
    }

import math
parameters_dict = {
    'n_steps':{
        'values':[2048, 256, 128,512,1024,4098]
        },
    'total_timesteps':{
        'values':[50000, 100000, 200000,300000,500000]
        },
    'ent_coef': {
                'distribution': 'uniform',
        'min': 0,
        'max': 0.1
        },
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
      },
    'batch_size': {
        # integers between 32 and 256
        # with evenly-distributed logarithms 
        'distribution': 'q_log_uniform',
        'q': 1,
        'min': math.log(32),
        'max': math.log(1024),
      }
    }

sweep_config['parameters'] = parameters_dict








TRAIN_START_DATE = '2008-01-01'
TRAIN_END_DATE = '2020-03-01'
TRADE_START_DATE = '2020-03-01'
TRADE_END_DATE = '2022-10-10'


res = pd.read_csv('DOW_8s.csv')   #读取整理好后数据，根据情况进行时间格式转换
#INDICATORS.pop()
INDICATORS
INDICATORS.append('sentiment_score')


#时间格式转换
res['date'] = pd.to_datetime(res['date'],errors='coerce')     #str转date
res['date'] = res['date'].dt.strftime('%Y-%m-%d')   #date转str


train = data_split(res, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(res, TRADE_START_DATE,TRADE_END_DATE)


#建议环境配置
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 20,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}


e_train_gym = StockTradingEnv(df = train, **env_kwargs)


#设置train环境
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))
sweep_id = wandb.sweep(sweep_config, project="test-project")


def train():
    wandb.init()
    agent = DRLAgent(env = env_train)
    PPO_PARAMS = {
            "ent_coef": wandb.config.ent_coef,
            "n_steps": wandb.config.n_steps,
            "learning_rate": wandb.config.learning_rate,
            "batch_size": wandb.config.batch_size,}


    model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)




    trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=wandb.config.total_timesteps)




    #Set turbulence threshold
    data_risk_indicator = res[(res.date<TRAIN_END_DATE) & (res.date>=TRAIN_START_DATE)]
    insample_risk_indicator = data_risk_indicator.drop_duplicates(subset=['date'])
    insample_risk_indicator.vix.describe() #VIX
    insample_risk_indicator.vix.quantile(0.996)
    insample_risk_indicator.turbulence.describe() #turbulence
    insample_risk_indicator.turbulence.quantile(0.996)



    e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70,risk_indicator_col='vix', **env_kwargs)
    trained_moedl = trained_ppo
    df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=trained_moedl, 
    environment = e_trade_gym)
    #df_account_value.shape #资产格式
    #print(df_account_value.tail()) #资产状态
    #df_actions.tail(200)    #交易操作



    print("==============Get Backtest Results===========")
    now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

    perf_stats_all = backtest_stats(account_value=df_account_value)
    perf_stats_all = pd.DataFrame(perf_stats_all)



    wandb.log({"Backtest Results": perf_stats_all,
          "action": df_actions,})
    
    
wandb.agent(sweep_id, train, count=50)

wandb.finish()

/Users/zhangzhilu/opt/anaconda3/envs/finrl/lib/python3.7/site-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; mutltipliers will not be applied' +
wandb: Currently logged in as: zhilu (finsrl). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/zhangzhilu/.netrc


             date   tic        open        high         low       close  \
0      2008-01-02  AAPL    7.116786    7.152143    6.876786    5.931611   
1      2008-01-02   CAT   72.559998   72.669998   70.050003   46.539093   
2      2008-01-02   DIS   32.320000   32.630001   31.690001   27.012506   
3      2008-01-02    GS  214.800003  215.050003  206.600006  166.547913   
4      2008-01-02  INTC   26.280001   26.340000   24.950001   16.023220   
...           ...   ...         ...         ...         ...         ...   
24483  2020-02-28    GS  199.089996  202.250000  194.850006  190.099899   
24484  2020-02-28  INTC   54.299999   56.380001   53.599998   51.038109   
24485  2020-02-28   JPM  117.519997  118.489998  112.660004  106.632568   
24486  2020-02-28   MMM  154.089996  156.720001  146.000000  136.575607   
24487  2020-02-28  MSFT  152.410004  163.710007  152.000000  158.378555   

             volume  day      macd     boll_ub     boll_lb      rsi_30  \
0      1.079179e+09  2.0 

wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. batch_size uses q_log_uniform, where min/max specify base-e exponents. Use q_log_uniform_values to specify limit values.


Create sweep with ID: wjkvqlsd
Sweep URL: https://wandb.ai/finsrl/test-project/sweeps/wjkvqlsd


wandb: Agent Starting Run: ufb9b2fs with config:
wandb: 	batch_size: 556
wandb: 	ent_coef: 0.010850762418435711
wandb: 	learning_rate: 0.04205927886524091
wandb: 	n_steps: 128
wandb: 	total_timesteps: 300000


...

-----------------------------------------
| time/                   |             |
|    fps                  | 79          |
|    iterations           | 22          |
|    time_elapsed         | 566         |
|    total_timesteps      | 45056       |
| train/                  |             |
|    approx_kl            | 0.094988294 |
|    clip_fraction        | 0.67        |
|    clip_range           | 0.2         |
|    entropy_loss         | -34.2       |
|    explained_variance   | 0.292       |
|    learning_rate        | 0.0372      |
|    loss                 | 13          |
|    n_updates            | 210         |
|    policy_gradient_loss | 0.0677      |
|    reward               | -2.6540387  |
|    std                  | 21.7        |
|    value_loss           | 113         |
-----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 80          |
|    iterations           | 23    

-----------------------------------------
| time/                   |             |
|    fps                  | 85          |
|    iterations           | 32          |
|    time_elapsed         | 763         |
|    total_timesteps      | 65536       |
| train/                  |             |
|    approx_kl            | 0.09317931  |
|    clip_fraction        | 0.404       |
|    clip_range           | 0.2         |
|    entropy_loss         | -35.8       |
|    explained_variance   | 0.84        |
|    learning_rate        | 0.0372      |
|    loss                 | 5.94        |
|    n_updates            | 310         |
|    policy_gradient_loss | 0.033       |
|    reward               | -0.31424832 |
|    std                  | 26.5        |
|    value_loss           | 20.5        |
-----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 86          |
|    iterations           | 33    

-----------------------------------------
| time/                   |             |
|    fps                  | 93          |
|    iterations           | 42          |
|    time_elapsed         | 924         |
|    total_timesteps      | 86016       |
| train/                  |             |
|    approx_kl            | 0.09294591  |
|    clip_fraction        | 0.605       |
|    clip_range           | 0.2         |
|    entropy_loss         | -37.6       |
|    explained_variance   | 0.647       |
|    learning_rate        | 0.0372      |
|    loss                 | 11.3        |
|    n_updates            | 410         |
|    policy_gradient_loss | 0.0661      |
|    reward               | -0.57562315 |
|    std                  | 33.6        |
|    value_loss           | 99.1        |
-----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 93          |
|    iterations           | 43    

----------------------------------------
| time/                   |            |
|    fps                  | 100        |
|    iterations           | 52         |
|    time_elapsed         | 1057       |
|    total_timesteps      | 106496     |
| train/                  |            |
|    approx_kl            | 0.03743129 |
|    clip_fraction        | 0.365      |
|    clip_range           | 0.2        |
|    entropy_loss         | -39.4      |
|    explained_variance   | 0.552      |
|    learning_rate        | 0.0372     |
|    loss                 | 21.9       |
|    n_updates            | 510        |
|    policy_gradient_loss | 0.0255     |
|    reward               | 0.18828982 |
|    std                  | 43         |
|    value_loss           | 98.1       |
----------------------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 101        |
|    iterations           | 53         |
|    time_elapse

-----------------------------------------
| time/                   |             |
|    fps                  | 109         |
|    iterations           | 62          |
|    time_elapsed         | 1157        |
|    total_timesteps      | 126976      |
| train/                  |             |
|    approx_kl            | 0.010724747 |
|    clip_fraction        | 0.303       |
|    clip_range           | 0.2         |
|    entropy_loss         | -41.6       |
|    explained_variance   | 0.881       |
|    learning_rate        | 0.0372      |
|    loss                 | 5.03        |
|    n_updates            | 610         |
|    policy_gradient_loss | 0.0142      |
|    reward               | 1.3609204   |
|    std                  | 52.4        |
|    value_loss           | 20.2        |
-----------------------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 110        |
|    iterations           | 63       

In [2]:
print(perf_stats_all)

                            0
Annual return        0.021349
Cumulative returns   0.039661
Annual volatility    0.066063
Sharpe ratio         0.353407
Calmar ratio         0.201586
Stability            0.425234
Max drawdown        -0.105903
Omega ratio          1.070212
Sortino ratio        0.517351
Skew                      NaN
Kurtosis                  NaN
Tail ratio           1.169491
Daily value at risk -0.008231


In [11]:
#trading



hit end!
           date  account_value
459  2021-12-23   1.140401e+06
460  2021-12-27   1.151409e+06
461  2021-12-28   1.153059e+06
462  2021-12-29   1.153755e+06
463  2021-12-30   1.151230e+06


,AAPL,CAT,DIS,GS,INTC,JPM,MMM,MSFT
date,,,,,,,,
2021-03-17,4,0,3,0,0,0,2,1
2021-03-18,4,0,3,0,0,0,2,1
2021-03-19,4,0,3,0,0,0,2,1
2021-03-22,4,0,3,0,0,0,2,1
2021-03-23,4,0,3,0,0,0,2,1
...,...,...,...,...,...,...,...,...
2021-12-22,4,0,3,0,0,0,2,1
2021-12-23,4,0,3,0,0,0,2,1
2021-12-27,4,0,3,0,0,0,2,1


In [12]:
#Backtesting Results
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_"+now+'.csv')

==============Get Backtest Results===========
Annual return          0.079487
Cumulative returns     0.151230
Annual volatility      0.063709
Sharpe ratio           1.235131
Calmar ratio           1.649271
Stability              0.929493
Max drawdown          -0.048195
Omega ratio            1.263219
Sortino ratio          1.801034
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.026593
Daily value at risk   -0.007714
dtype: float64


In [13]:
#df_actions.to_csv('action_ppo_s(h20)3.csv')


In [14]:
#a = perf_stats_all.to_dict()
#wandb.save(perf_stats_all)
#wandb.save(df_actions)
